# Rotational Trading

Rotational trading involves purchasing the best-performing assets while selling underperforming ones. As you may have guessed, **PyBroker** is an excellent tool for backtesting such strategies. So, let's dive in and get started with testing our rotational trading strategy!

In [1]:
import pybroker as pyb
from pybroker import ExecContext, Strategy, StrategyConfig, YFinance

Our strategy will involve ranking and buying stocks with the highest [price rate-of-change (ROC)](https://www.investopedia.com/terms/p/pricerateofchange.asp). To start, we'll define a 20-day ROC indicator using [TA-Lib](https://github.com/TA-Lib/ta-lib-python):

In [2]:
import talib as ta

roc_20 = pyb.indicator(
    'roc_20', lambda data: ta.ROC(data.adj_close, timeperiod=20))

Next, let's define the rules of our strategy:

- Buy the two stocks with the highest 20-day ROC.
- Allocate 50% of our capital to each stock.
- If either of the stocks is no longer ranked among the top five 20-day ROCs, then we will liquidate that stock.
- Trade these rules daily.

Let’s set up our config and some parameters for the above rules:

In [3]:
config = StrategyConfig(max_long_positions=2)
pyb.param('target_size', 1 / config.max_long_positions)
pyb.param('rank_threshold', 5)

5

To proceed with our strategy, we will implement a ``rank`` function that ranks each stock by their 20-day ROC in descending order, from highest to lowest.

In [4]:
def rank(ctxs: dict[str, ExecContext]):
    scores = {
        symbol: ctx.indicator('roc_20')[-1]
        for symbol, ctx in ctxs.items()
    }
    sorted_scores = sorted(
        scores.items(), 
        key=lambda score: score[1],
        reverse=True
    )
    threshold = pyb.param('rank_threshold')
    top_scores = sorted_scores[:threshold]
    top_symbols = [score[0] for score in top_scores]
    pyb.param('top_symbols', top_symbols)

The ``top_symbols`` global parameter contains the symbols of the stocks with the top five highest 20-day ROCs.

Now that we have a method for ranking stocks by their ROC, we can proceed with implementing a ``rotate`` function to manage the rotational trading.

In [5]:
def rotate(ctx: ExecContext):
    if ctx.long_pos():
        if ctx.symbol not in pyb.param('top_symbols'):
            ctx.sell_all_shares()
    else:
        target_size = pyb.param('target_size')
        ctx.buy_shares = ctx.calc_target_shares(target_size)
        ctx.score = ctx.indicator('roc_20')[-1]

We liquidate the currently held stock if it is no longer ranked among the top five 20-day ROCs. Otherwise, we rank all stocks by their 20-day ROC and buy up to the top two ranked. For more information on ranking when placing buy orders, see the [Ranking and Position Sizing notebook](https://www.pybroker.com/en/latest/notebooks/4.%20Ranking%20and%20Position%20Sizing.html).

We will use the [set_before_exec](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.set_before_exec) method to execute our ranking with ``rank`` before running the ``rotate`` function. For this backtest, we will use a universe of 10 stocks:

In [6]:
strategy = Strategy(
    YFinance(), 
    start_date='1/1/2018', 
    end_date='1/1/2023', 
    config=config
)
strategy.set_before_exec(rank)
strategy.add_execution(rotate, [
    'TSLA', 
    'NFLX', 
    'AAPL', 
    'NVDA', 
    'AMZN',
    'MSFT',
    'GOOG',
    'AMD',
    'INTC',
    'META'
], indicators=roc_20)
result = strategy.backtest()

Backtesting: 2018-01-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  10 of 10 completed
Loaded bar data: 0:00:01 

Computing indicators...


100% (10 of 10) |########################| Elapsed Time: 0:00:00 Time:  0:00:00



Test split: 2018-01-02 00:00:00 to 2022-12-30 00:00:00


100% (1259 of 1259) |####################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:04


In [7]:
result.orders

,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,buy,AAPL,2018-01-03,1161,NaN,43.31,0.0
2,buy,AMD,2018-01-03,4231,NaN,11.75,0.0
3,sell,AAPL,2018-02-01,1161,NaN,41.92,0.0
4,buy,NFLX,2018-02-01,181,NaN,267.67,0.0
5,sell,AMD,2018-02-05,4231,NaN,11.56,0.0
...,...,...,...,...,...,...,...
258,buy,AMD,2022-11-21,3764,NaN,72.28,0.0
259,sell,AMD,2022-12-14,3764,NaN,70.16,0.0
260,buy,NFLX,2022-12-14,924,NaN,319.57,0.0
